# RL Zeittabelle

Dieser Code erstellt eine Zeittabelle für verschiedene Kostenträger. 
Nach Angabe eines Monats und eines Jahres werden automatisch die Arbeitstage dieses Monats bestimmt. Anschließend wird eine reguläre Arbeitszeit abgefragt. Für Tage an denen nicht die reguläre Zeit gearbeitet wurde kann individuell die Arbeitszeit angegeben werden.
Anschließend wird eine Liste der Kostenträger (KT) abgefragt und der nominelle relative Arbeitsanteil je Kostenträger abefragt. Verkürzte Angaben werdend dabei automatisch zu validen KT erweitert (z.B. '3' zu '3025503'). Der Arbeitsanteil des letzten KTs der Liste wird automatisch berechnet um auf 100% Arbeitszeitverteilung zu kommen.
Das Programm ermittelt dann zufällig für jeden Tag eine Arbeitszeit je KT so dass die angegebene Gesamtarbeitszeit je Tag erreicht wird. (Die Zeiten der einzelnen KT folgen dabei einer Normalverteilung mit einer Schwankungsbreite von einer halben Stunde.) Das Programm ist so geschrieben, dass die angegebenen Arbeitszeiten je Tag exakt getroffen werden, der reale Arbeitsanteil je KT für den Gesamtmonat jedoch von dem Soll-Wert leicht abweichen kann. 
Am Ende wird eine Tabelle mit den Arbeitstagen, der Gesamt-Arbeitszeit je Tag sowie der Arbeitszeit je KT ausgegeben. Außerdem wird die Abweichung vom Soll-Anteil über den gesamten Monat ermittelt.
Zum Ausführen einfach den "Run" Knopf oben in der Leiste klicken und in den Eingabefenstern unterhalb des Codes die abgefragten Informationen eingeben.

In [7]:
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 14 13:52:11 2020

@author: scha_su
"""

import calendar
import sys
import random

month = int(input("Bitte Monat eingeben (als Zahl) "))
year = int(input("Bitte Jahr eingeben (vollständig) "))

cal= calendar.Calendar()

workdays = []
for x in cal.itermonthdays2(year, month):
    if x[0] != 0 and x[1] < 5:
        workdays.append(x[0])
print()
row_format ="{:>5}" * (len(workdays))
print ("In " + str(month) + '.' + str(year) +' gab es folgende Arbeitstage:')
print (row_format.format(*workdays))        

regwh = float(input("Bitte reguläre Arbeitszeit eingeben (als Dezimalwert, Standard: 7.8) ").replace(',','.') or '7.8')
s = input("Bitte Tage eingeben, an denen nicht die reguläre Stundenzahl (" + str(regwh) +") gearbeitet wurde (mit Leerzeichen getrennt) ")
nonregdays = list(map(int, s.split()))

workhours = []
for day in workdays:
    if day in nonregdays: 
        wh = float(input("Bitte Arbeitszeit eingeben für den "+str(day)+'.'+str(month)+'.'+str(year)+ ' '))
        workhours.append(wh)
    else:
        workhours.append(regwh)
        
# print (workhours)

kts = input("Bitte Liste der Kostenträger eingeben (mit Leerzeichen getrennt) ").split() or ['3025502', '3025503', '3025504']
for kti in range(len(kts)):
    if len(kts[kti]) == 1:
        kts[kti] = '302550' + kts[kti]
    elif len(kts[kti]) == 2:
        kts[kti] = '30255' + kts[kti]
frac = []
maxdig = 0
for kt in kts[:-1]: 
    t = input("Bitte nominellen Arbeitsanteil für Kostenträger " + str(kt) + " angeben (dezimal) ").replace(',','.')
    frac.append(float(t))
    if len(t.split(".")[-1]) > maxdig: maxdig = len(t.split(".")[-1])
frac.append(round(1.0-sum(frac), maxdig))
print("Nomineller Arbeitsanteil für Kostenträger " + str(kts[-1]) + ": " + str(frac[-1])) 

# print (kts, frac)
    
sigma = .5

kttimes = []
for kt in kts:
    kttimes.append([])

for day in range(len(workhours)):
    totlen = 0
    if workhours[day] == 0:
        for kti in range(len(kts)):
            kttimes[kti].append(0)
        continue
    for kti in range(len(kts)-1):
        time = round(random.gauss(frac[kti]*workhours[day], sigma),1)
        if time < 0: time = 0
        kttimes[kti].append(time)
        totlen += kttimes[kti][day]
    kttimes[-1].append(round(workhours[day]-totlen,1))
    
totwh = sum(workhours)
print()
print('Arbeitstage\t',row_format.format(*workdays))
print('Arbeitszeit\t',row_format.format(*workhours))

worktime = []
for kti in range(len(kts)):
    print('KT '+kts[kti]+'\t', row_format.format(*kttimes[kti]))
    worktime.append(sum(kttimes[kti]))
print()
print('Arbeitsanteil je KT')
for kti in range(len(kts)):
    print('KT ' + kts[kti], str(round(worktime[kti]/totwh*100,1))+'%')


Bitte Monat eingeben (als Zahl) 8
Bitte Jahr eingeben (vollständig) 2020

In 8.2020 gab es folgende Arbeitstage:
    3    4    5    6    7   10   11   12   13   14   17   18   19   20   21   24   25   26   27   28   31
Bitte reguläre Arbeitszeit eingeben (als Dezimalwert, Standard: 7.8) 
Bitte Tage eingeben, an denen nicht die reguläre Stundenzahl (7.8) gearbeitet wurde (mit Leerzeichen getrennt) 
Bitte Liste der Kostenträger eingeben (mit Leerzeichen getrennt) 
Bitte nominellen Arbeitsanteil für Kostenträger 3025502 angeben (dezimal) ,1
Bitte nominellen Arbeitsanteil für Kostenträger 3025503 angeben (dezimal) ,2
Nomineller Arbeitsanteil für Kostenträger 3025504: 0.7

Arbeitstage	     3    4    5    6    7   10   11   12   13   14   17   18   19   20   21   24   25   26   27   28   31
Arbeitszeit	   7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8  7.8
KT 3025502	   0.9  0.2  0.7  0.7  0.8  1.2  0.6  0.7  1.5  1.0  0.9  2.0  1.0  1.4  0